In [64]:
import os,glob
import json
import copy
import numpy as np
import pandas as pd

# from sklearn.metrics import pairwise_distances
# from scipy.spatial import distance_matrix
# from scipy.spatial.distance import cdist
import skimage.morphology

# import matplotlib.pyplot as plt
# from matplotlib.colors import LogNorm

import SimpleITK as sitk
from file_io import load_image,save_image



In [65]:
def get_tree():
    with open('tree_yzx.json','r') as f:
        tree = json.loads(f.read())
    return tree

def get_tree_from(info,hold=[]):
    info_tree = {}
    tree = get_tree()
    for t in tree:
        if (not hold) or (t[info] in hold):
            info_tree[t[info]] = t  
    return info_tree

def write_json(write_dict,dict_file):
    dict_object = json.dumps(write_dict)
    with open(dict_file, "w") as f:
        f.write(dict_object)

def load_json(dict_file):
    with open(dict_file, "r") as f:
        load_dict = json.loads(f.read())
    return load_dict

def load_json_key2int(dict_file):
    with open(dict_file, "r") as f:
        load_dict = json.loads(f.read())
    new_dict = {}
    for k,v in load_dict.items():
        new_dict[int(k)] = v
    return new_dict

def load_json_keys2int(dict_file):
    with open(dict_file, "r") as f:
        load_dict = json.loads(f.read())
    new_dict = {}
    for k,v in load_dict.items():
        new_dict[k] = {}
        for kk,vv in v.items():
            new_dict[k][int(kk)] = vv
    return new_dict

def load_level_csv():
    level_file = 'level.csv'
    level_df = pd.read_csv(level_file)
    return level_df

# produce u32 level csv

In [66]:
region_tree = get_tree_from('acronym')
print(region_tree['root']['id'],region_tree['root'])
print(region_tree['fiber tracts']['id'],region_tree['fiber tracts'])
print(region_tree['VS']['id'],region_tree['VS'])
print(region_tree['grv']['id'],region_tree['grv'])
print(region_tree['retina']['id'],region_tree['retina'])
print(region_tree['CH']['id'],region_tree['CH'])

997 {'acronym': 'root', 'rgb_triplet': [255, 255, 255], 'graph_id': 1, 'graph_order': 0, 'id': 997, 'name': 'root', 'structure_id_path': [997], 'structure_set_ids': [691663206]}
1009 {'acronym': 'fiber tracts', 'rgb_triplet': [204, 204, 204], 'graph_id': 1, 'graph_order': 1101, 'id': 1009, 'name': 'fiber tracts', 'structure_id_path': [997, 1009], 'structure_set_ids': [687527945, 184527634, 167587189, 691663206]}
73 {'acronym': 'VS', 'rgb_triplet': [170, 170, 170], 'graph_id': 1, 'graph_order': 1292, 'id': 73, 'name': 'ventricular systems', 'structure_id_path': [997, 73], 'structure_set_ids': [691663206]}
1024 {'acronym': 'grv', 'rgb_triplet': [170, 170, 170], 'graph_id': 1, 'graph_order': 1304, 'id': 1024, 'name': 'grooves', 'structure_id_path': [997, 1024], 'structure_set_ids': []}
304325711 {'acronym': 'retina', 'rgb_triplet': [127, 46, 126], 'graph_id': 1, 'graph_order': 1326, 'id': 304325711, 'name': 'retina', 'structure_id_path': [997, 304325711], 'structure_set_ids': [114512891, 

In [67]:
n4_region_tree = get_tree_from('acronym',['CTX','CNU','CB','BS'])
print(len(n4_region_tree),n4_region_tree)
n4_u32_id_list = [t['id'] for t in n4_region_tree.values()]
print(len(n4_u32_id_list),n4_u32_id_list)    

4 {'CTX': {'acronym': 'CTX', 'rgb_triplet': [176, 255, 184], 'graph_id': 1, 'graph_order': 3, 'id': 688, 'name': 'Cerebral cortex', 'structure_id_path': [997, 8, 567, 688], 'structure_set_ids': [112905828, 691663206, 12, 184527634, 112905813, 114512891, 114512892]}, 'CNU': {'acronym': 'CNU', 'rgb_triplet': [152, 214, 249], 'graph_id': 1, 'graph_order': 570, 'id': 623, 'name': 'Cerebral nuclei', 'structure_id_path': [997, 8, 567, 623], 'structure_set_ids': [112905828, 691663206, 12, 184527634, 112905813, 114512891, 114512892]}, 'BS': {'acronym': 'BS', 'rgb_triplet': [255, 112, 128], 'graph_id': 1, 'graph_order': 639, 'id': 343, 'name': 'Brain stem', 'structure_id_path': [997, 8, 343], 'structure_set_ids': [112905828, 691663206, 12, 184527634, 112905813, 114512891, 114512892]}, 'CB': {'acronym': 'CB', 'rgb_triplet': [240, 240, 128], 'graph_id': 1, 'graph_order': 1014, 'id': 512, 'name': 'Cerebellum', 'structure_id_path': [997, 8, 512], 'structure_set_ids': [2, 112905828, 691663206, 12, 1

In [68]:
n13_region_list = ['Isocortex', 'OLF', 'HPF', 'CTXsp', 'STR', 'PAL', 'TH', 'HY', 'MB', 'P', 'MY', 'CBX', 'CBN']
n13_u32_id_list = [region_tree[t]['id'] for t in n13_region_list]
n13_u32_id_list

[315, 698, 1089, 703, 477, 803, 549, 1097, 313, 771, 354, 528, 519]

In [69]:
n316_u32_id_list = pd.read_csv('CCFv3_316son_70parent_structures.csv')['structure ID'].tolist()
for tmp in [1009]:
    tmp = n316_u32_id_list.index(tmp)
    n316_u32_id_list.pop(tmp)
print(len(n316_u32_id_list),n316_u32_id_list)

315 [184, 985, 993, 353, 329, 337, 345, 369, 361, 182305689, 378, 1057, 677, 1011, 1002, 1027, 1018, 402, 394, 409, 385, 425, 533, 312782574, 312782628, 39, 48, 972, 44, 723, 731, 746, 104, 111, 119, 894, 879, 886, 312782546, 417, 541, 922, 895, 507, 151, 159, 589, 814, 961, 619, 639, 647, 788, 566, 382, 423, 463, 726, 982, 19, 918, 926, 843, 1037, 1084, 502, 484682470, 589508447, 484682508, 583, 952, 966, 131, 295, 319, 780, 672, 56, 998, 754, 250, 258, 266, 310, 333, 23, 292, 536, 1105, 403, 1022, 1031, 342, 298, 564, 596, 581, 351, 287, 629, 685, 718, 725, 733, 741, 563807435, 414, 422, 609, 1044, 475, 170, 218, 1020, 1029, 325, 255, 127, 64, 1120, 1113, 155, 59, 362, 366, 1077, 149, 15, 181, 560581559, 189, 599, 907, 575, 930, 560581563, 262, 27, 563807439, 178, 321, 483, 186, 390, 332, 38, 30, 118, 223, 72, 263, 272, 830, 452, 523, 763, 914, 1109, 126, 133, 347, 286, 338, 576073699, 689, 88, 210, 491, 525, 1126, 1, 515, 980, 1004, 63, 693, 946, 194, 226, 356, 364, 576073704, 173, 

In [74]:
n671_u32_mask = load_image("annotation_25.nrrd")#z456, y320, x528
n671_u32_id_list,n671_u32_id_count_list = np.unique(n671_u32_mask,return_counts=True)
n671_u32_id_list = n671_u32_id_list.tolist()
n671_u32_id_count_list = n671_u32_id_count_list.tolist()
for tmp in [1009,997,0]:
    tmp = n671_u32_id_list.index(tmp)
    n671_u32_id_list.pop(tmp)
    n671_u32_id_count_list.pop(tmp)   
print(len(n671_u32_id_list),n671_u32_id_list)

new_file = 'n671_u32.nrrd'
new_mask = n671_u32_mask.copy() 
new_mask = new_mask.astype(np.uint32)
for n671_u32_id in [73,567,997,1009,1024,304325711]:
    new_mask[n671_u32_mask==n671_u32_id] = 0
sitk.WriteImage(sitk.GetImageFromArray(new_mask), new_file)   

669 [1, 2, 6, 7, 9, 10, 12, 15, 17, 19, 20, 23, 26, 27, 28, 30, 33, 35, 36, 38, 41, 42, 50, 52, 54, 56, 58, 59, 62, 63, 64, 66, 67, 68, 72, 74, 75, 78, 81, 83, 84, 88, 91, 93, 96, 97, 98, 100, 101, 102, 105, 106, 108, 113, 114, 115, 117, 118, 120, 121, 122, 123, 125, 126, 128, 129, 131, 132, 133, 136, 139, 140, 143, 145, 146, 147, 148, 149, 153, 155, 156, 158, 159, 162, 163, 164, 169, 171, 173, 177, 178, 180, 181, 186, 187, 188, 189, 190, 194, 196, 197, 198, 201, 202, 203, 204, 206, 207, 209, 210, 211, 214, 215, 217, 218, 222, 223, 225, 226, 229, 230, 231, 233, 234, 237, 238, 243, 246, 249, 250, 251, 252, 255, 257, 258, 260, 262, 263, 266, 268, 269, 271, 272, 274, 279, 280, 281, 286, 287, 288, 289, 292, 296, 298, 301, 303, 304, 305, 307, 310, 311, 313, 314, 318, 320, 321, 325, 326, 327, 328, 330, 332, 333, 334, 335, 336, 338, 342, 344, 347, 349, 350, 351, 354, 355, 356, 358, 362, 363, 364, 366, 368, 372, 374, 377, 380, 381, 382, 390, 393, 397, 401, 403, 412, 413, 414, 421, 422, 423, 42

In [75]:
u32_id_tree = get_tree_from('id')
print(len(u32_id_tree))
n1327_u32_id_list = list(u32_id_tree.keys())
print(len(n1327_u32_id_list),n1327_u32_id_list)

1327
1327 [997, 8, 567, 688, 695, 315, 184, 68, 667, 526157192, 526157196, 526322264, 500, 107, 219, 299, 644, 947, 985, 320, 943, 648, 844, 882, 993, 656, 962, 767, 1021, 1085, 453, 12993, 12994, 12995, 12996, 12997, 12998, 322, 793, 346, 865, 921, 686, 719, 353, 558, 838, 654, 702, 889, 929, 329, 981, 201, 1047, 1070, 1038, 1062, 480149202, 480149206, 480149210, 480149214, 480149218, 480149222, 480149226, 337, 1030, 113, 1094, 1128, 478, 510, 345, 878, 657, 950, 974, 1102, 2, 369, 450, 854, 577, 625, 945, 1026, 361, 1006, 670, 1086, 1111, 9, 461, 182305689, 182305693, 182305697, 182305701, 182305705, 182305709, 182305713, 378, 873, 806, 1035, 1090, 862, 893, 1057, 36, 180, 148, 187, 638, 662, 677, 897, 1106, 1010, 1058, 857, 849, 247, 1011, 527, 600, 678, 252, 156, 243, 480149230, 480149234, 480149238, 480149242, 480149246, 480149250, 480149254, 1002, 735, 251, 816, 847, 954, 1005, 1027, 696, 643, 759, 791, 249, 456, 1018, 959, 755, 990, 1023, 520, 598, 669, 801, 561, 913, 937, 457, 

In [76]:
level_csv = []
level_file = 'level.csv'
for n1327_u32_id,t in u32_id_tree.items():
    tmp = t['structure_id_path']
 
    tmp0 = list(set(tmp).intersection(set(n4_u32_id_list)))
    if not tmp0: tmp0 = 0
    else: 
        if len(tmp0)>1: raise
        tmp0 = tmp0[0]
    
        
    tmp1 = list(set(tmp).intersection(set(n13_u32_id_list)))
    if not tmp1: #688 695 623 343 1129 1065 512 CTX CTXpl CNU BS IB HB CB 
        tmp1 = 0
    else: 
        if len(tmp1)>1: raise
        tmp1 = tmp1[0]
    
    tmp2 = list(set(tmp).intersection(set(n316_u32_id_list)))
    if not tmp2: tmp2 = 0
    else: tmp2 = [tm for tm in tmp if tm in tmp2][-1]
    #raise 395/1098 395/1107
    
    tmp3 = list(set(tmp).intersection(set(n671_u32_id_list)))
    if not tmp3: tmp3 = 0
    else: tmp3 = [tm for tm in tmp if tm in tmp3][-1]
        #raise 
        #159 160 167 168 175 183 188 191 196 199 204 297 507 597 1034 1042 1050 ...
        #[1097, 797, 804][313, 795,][313,100][771, 867, 123][1009]

    print(t['graph_order'],n1327_u32_id,tmp,tmp3,tmp2,tmp1,tmp0)
    level_csv.append([n1327_u32_id,tmp3,tmp2,tmp1,tmp0])
level_df = pd.DataFrame(np.array(level_csv),columns=['n1327_u32_id','n671_u32_id','n316_u32_id','n13_u32_id','n4_u32_id'])
level_df.to_csv(level_file, index=False)
level_df

0 997 [997] 0 0 0 0
1 8 [997, 8] 0 0 0 0
2 567 [997, 8, 567] 0 0 0 0
3 688 [997, 8, 567, 688] 0 0 0 688
4 695 [997, 8, 567, 688, 695] 0 0 0 688
5 315 [997, 8, 567, 688, 695, 315] 0 0 315 688
6 184 [997, 8, 567, 688, 695, 315, 184] 0 184 315 688
7 68 [997, 8, 567, 688, 695, 315, 184, 68] 68 184 315 688
8 667 [997, 8, 567, 688, 695, 315, 184, 667] 667 184 315 688
9 526157192 [997, 8, 567, 688, 695, 315, 184, 526157192] 526157192 184 315 688
10 526157196 [997, 8, 567, 688, 695, 315, 184, 526157196] 526157196 184 315 688
11 526322264 [997, 8, 567, 688, 695, 315, 184, 526322264] 526322264 184 315 688
12 500 [997, 8, 567, 688, 695, 315, 500] 0 0 315 688
13 107 [997, 8, 567, 688, 695, 315, 500, 107] 0 0 315 688
14 219 [997, 8, 567, 688, 695, 315, 500, 219] 0 0 315 688
15 299 [997, 8, 567, 688, 695, 315, 500, 299] 0 0 315 688
16 644 [997, 8, 567, 688, 695, 315, 500, 644] 0 0 315 688
17 947 [997, 8, 567, 688, 695, 315, 500, 947] 0 0 315 688
18 985 [997, 8, 567, 688, 695, 315, 500, 985] 0 985 31

323 480149334 [997, 8, 567, 688, 695, 315, 254, 894, 480149314, 480149334] 0 894 315 688
324 480149338 [997, 8, 567, 688, 695, 315, 254, 894, 480149314, 480149338] 0 894 315 688
325 879 [997, 8, 567, 688, 695, 315, 254, 879] 0 879 315 688
326 442 [997, 8, 567, 688, 695, 315, 254, 879, 442] 442 879 315 688
327 434 [997, 8, 567, 688, 695, 315, 254, 879, 434] 434 879 315 688
328 545 [997, 8, 567, 688, 695, 315, 254, 879, 545] 0 879 315 688
329 610 [997, 8, 567, 688, 695, 315, 254, 879, 610] 610 879 315 688
330 274 [997, 8, 567, 688, 695, 315, 254, 879, 274] 274 879 315 688
331 330 [997, 8, 567, 688, 695, 315, 254, 879, 330] 330 879 315 688
332 886 [997, 8, 567, 688, 695, 315, 254, 886] 0 886 315 688
333 542 [997, 8, 567, 688, 695, 315, 254, 886, 542] 542 886 315 688
334 606 [997, 8, 567, 688, 695, 315, 254, 886, 606] 0 886 315 688
335 430 [997, 8, 567, 688, 695, 315, 254, 886, 430] 430 886 315 688
336 687 [997, 8, 567, 688, 695, 315, 254, 886, 687] 687 886 315 688
337 590 [997, 8, 567, 68

659 1072 [997, 8, 343, 1129, 549, 864, 1008, 475, 1072] 1072 475 549 343
660 1079 [997, 8, 343, 1129, 549, 864, 1008, 475, 1079] 1079 475 549 343
661 1088 [997, 8, 343, 1129, 549, 864, 1008, 475, 1088] 1088 475 549 343
662 170 [997, 8, 343, 1129, 549, 864, 1008, 170] 549 170 549 343
663 496345664 [997, 8, 343, 1129, 549, 864, 1008, 170, 496345664] 496345664 170 549 343
664 496345668 [997, 8, 343, 1129, 549, 864, 1008, 170, 496345668] 496345668 170 549 343
665 496345672 [997, 8, 343, 1129, 549, 864, 1008, 170, 496345672] 496345672 170 549 343
666 856 [997, 8, 343, 1129, 549, 856] 549 0 549 343
667 138 [997, 8, 343, 1129, 549, 856, 138] 549 0 549 343
668 218 [997, 8, 343, 1129, 549, 856, 138, 218] 218 218 549 343
669 1020 [997, 8, 343, 1129, 549, 856, 138, 1020] 1020 1020 549 343
670 1029 [997, 8, 343, 1129, 549, 856, 138, 1029] 1029 1029 549 343
671 325 [997, 8, 343, 1129, 549, 856, 138, 325] 325 325 549 343
672 560581551 [997, 8, 343, 1129, 549, 856, 138, 560581551] 560581551 0 549 343

1080 1025 [997, 8, 512, 528, 1073, 1025] 1025 1025 528 512
1081 10683 [997, 8, 512, 528, 1073, 1025, 10683] 1025 1025 528 512
1082 10682 [997, 8, 512, 528, 1073, 1025, 10682] 1025 1025 528 512
1083 10681 [997, 8, 512, 528, 1073, 1025, 10681] 1025 1025 528 512
1084 1033 [997, 8, 512, 528, 1073, 1033] 1033 1033 528 512
1085 10686 [997, 8, 512, 528, 1073, 1033, 10686] 1033 1033 528 512
1086 10685 [997, 8, 512, 528, 1073, 1033, 10685] 1033 1033 528 512
1087 10684 [997, 8, 512, 528, 1073, 1033, 10684] 1033 1033 528 512
1088 1041 [997, 8, 512, 528, 1073, 1041] 1041 1041 528 512
1089 10689 [997, 8, 512, 528, 1073, 1041, 10689] 1041 1041 528 512
1090 10688 [997, 8, 512, 528, 1073, 1041, 10688] 1041 1041 528 512
1091 10687 [997, 8, 512, 528, 1073, 1041, 10687] 1041 1041 528 512
1092 1049 [997, 8, 512, 528, 1073, 1049] 1049 1049 528 512
1093 10692 [997, 8, 512, 528, 1073, 1049, 10692] 1049 1049 528 512
1094 10691 [997, 8, 512, 528, 1073, 1049, 10691] 1049 1049 528 512
1095 10690 [997, 8, 512, 52

n1327_u32_id  n671_u32_id  n316_u32_id  n13_u32_id  n4_u32_id
0              997            0            0           0          0
1                8            0            0           0          0
2              567            0            0           0          0
3              688            0            0           0        688
4              695            0            0           0        688
...            ...          ...          ...         ...        ...
1322            49            0            0           0          0
1323            57            0            0           0          0
1324            65            0            0           0          0
1325           624            0            0           0          0
1326     304325711            0            0           0          0

[1327 rows x 5 columns]

# produce u32 u16 dict

In [77]:
level_df = load_level_csv()

u32_u16_id_dict = {}
u32_u16_dict_file = 'u32_u16_dict.json'
u16_u32_id_dict = {}
u16_u32_dict_file = 'u16_u32_dict.json'
for tmp_i in level_df.index:
    u16_id_l = 2*tmp_i+1
    u16_id_r = 2*tmp_i+2
    u32_id = int(level_df.iloc[tmp_i]['n1327_u32_id'])
    u32_u16_id_dict[u32_id] = {'l':u16_id_l,'r':u16_id_r}
    u16_u32_id_dict[u16_id_l] = u32_id
    u16_u32_id_dict[u16_id_r] = u32_id
print(type(u32_id),type(u16_id_l),type(u16_id_r))
print(len(u32_u16_id_dict),u32_u16_id_dict)
write_json(u32_u16_id_dict,u32_u16_dict_file)
print(len(u16_u32_id_dict),u16_u32_id_dict)
write_json(u16_u32_id_dict,u16_u32_dict_file)

u32_u16_id_dict = load_json_key2int(u32_u16_dict_file)
for k,v in u32_u16_id_dict.items():
    print(type(k),type(v['l']),type(v['r']))
    break  
u16_u32_id_dict = load_json_key2int(u16_u32_dict_file)
for k,v in u32_u16_id_dict.items():
    print(type(k),type(v['l']),type(v['r']))
    break

<class 'int'> <class 'int'> <class 'int'>
1327 {997: {'l': 1, 'r': 2}, 8: {'l': 3, 'r': 4}, 567: {'l': 5, 'r': 6}, 688: {'l': 7, 'r': 8}, 695: {'l': 9, 'r': 10}, 315: {'l': 11, 'r': 12}, 184: {'l': 13, 'r': 14}, 68: {'l': 15, 'r': 16}, 667: {'l': 17, 'r': 18}, 526157192: {'l': 19, 'r': 20}, 526157196: {'l': 21, 'r': 22}, 526322264: {'l': 23, 'r': 24}, 500: {'l': 25, 'r': 26}, 107: {'l': 27, 'r': 28}, 219: {'l': 29, 'r': 30}, 299: {'l': 31, 'r': 32}, 644: {'l': 33, 'r': 34}, 947: {'l': 35, 'r': 36}, 985: {'l': 37, 'r': 38}, 320: {'l': 39, 'r': 40}, 943: {'l': 41, 'r': 42}, 648: {'l': 43, 'r': 44}, 844: {'l': 45, 'r': 46}, 882: {'l': 47, 'r': 48}, 993: {'l': 49, 'r': 50}, 656: {'l': 51, 'r': 52}, 962: {'l': 53, 'r': 54}, 767: {'l': 55, 'r': 56}, 1021: {'l': 57, 'r': 58}, 1085: {'l': 59, 'r': 60}, 453: {'l': 61, 'r': 62}, 12993: {'l': 63, 'r': 64}, 12994: {'l': 65, 'r': 66}, 12995: {'l': 67, 'r': 68}, 12996: {'l': 69, 'r': 70}, 12997: {'l': 71, 'r': 72}, 12998: {'l': 73, 'r': 74}, 322: {'

# produce u32 and u16 nrrd of level671 and level316

In [ ]:
def produce_nrrd(cl=671):
    n671_u32_file = f'n{cl}_u32.nrrd'  
    n671_u32_mask = sitk.GetArrayFromImage(sitk.ReadImage(n671_u32_file))
    n671_u32_mask = n671_u32_mask.astype(np.uint32)
    shape = n671_u32_mask.shape
    print(n671_u32_file,shape)

    n671_u16_file = f'n{cl}_u16.nrrd'
    n671_u16_mask = np.zeros(shape).astype(np.uint16)    
    
    cl_nl_dict = {671:316,316:0}
    nl = cl_nl_dict[cl]
    if not nl:
        print(f'n{cl}iter   index   n671_u32_id   n671_u16_id_l   n671_u16_id_r')
    else:
        n316_u32_file = f'n{nl}_u32.nrrd'
        n316_u32_mask = np.zeros(shape).astype(np.uint32)  
        print(f'n{cl}iter   n{nl}iter   index   n671_u32_id   n671_u16_id_l   n671_u16_id_r   n316_u32_id')
    
    level_file = 'level.csv'
    level_df = pd.read_csv(level_file)
    dict_file = 'u32_u16_dict.json'
    u32_u16_id_dict = load_json_key2int(dict_file)
    
    n671_u32_id_list = []
    if nl: n316_u32_id_list = []
    for i in level_df.index:
        n671_u32_id = level_df.iloc[i][f'n{cl}_u32_id']
        if (not n671_u32_id) or (n671_u32_id in n671_u32_id_list): continue
        else: n671_u32_id_list.append(n671_u32_id)
        n671_u16_id_l = u32_u16_id_dict[n671_u32_id]['l']
        n671_u16_id_r = u32_u16_id_dict[n671_u32_id]['r']
        
        mask_cur = n671_u32_mask == n671_u32_id
        mask_left = mask_cur.copy()
        mask_left[shape[0]//2:] = False
        n671_u16_mask[mask_left] = n671_u16_id_l
        mask_right = mask_cur.copy()
        mask_right[:shape[0]//2] = False   
        n671_u16_mask[mask_right] = n671_u16_id_r    
        
        if not nl:
            print(len(n671_u32_id_list),i,n671_u32_id,n671_u16_id_l,n671_u16_id_r)
        else:
            n316_u32_ids = np.unique(level_df[f'n{nl}_u32_id'][level_df[f'n{cl}_u32_id']==n671_u32_id]).tolist()
            if len(n316_u32_ids)>1: print(f'\n\n\n{n316_u32_ids}')
            n316_u32_ids.sort()
            n316_u32_id = n316_u32_ids[0]
            if not n316_u32_id in n316_u32_id_list: n316_u32_id_list.append(n316_u32_id)
            n316_u32_mask[mask_cur] = n316_u32_id       
            print(len(n671_u32_id_list),len(n316_u32_id_list),i,n671_u32_id,n671_u16_id_l,n671_u16_id_r,n316_u32_id)
    
    print(n671_u32_file)
    sitk.WriteImage(sitk.GetImageFromArray(n671_u16_mask), n671_u16_file)   
    if nl:
        print(n316_u32_file)
        sitk.WriteImage(sitk.GetImageFromArray(n316_u32_mask), n316_u32_file)  
    
produce_nrrd(cl=316)

# produce voxel_center_neighbor_dict of n671_u32 n671_u16 n316_u32 n316_u16

In [6]:
def get_center(mask):
    nz = np.nonzero(mask)
    zcoords, ycoords, xcoords = nz
    cz = zcoords.mean().round(2)
    cy = ycoords.mean().round(2)
    cx = xcoords.mean().round(2)
    return [float(cx), float(cy), float(cz)]


def produce_level(cl=671,num_voxels = 1):
    n671_u32_file = f'n{cl}_u32.nrrd'    
    n671_u32_mask = sitk.GetArrayFromImage(sitk.ReadImage(n671_u32_file))
    n671_u32_mask = n671_u32_mask.astype(np.uint32)
    n671_u16_file = f'n{cl}_u16.nrrd'    
    n671_u16_mask = sitk.GetArrayFromImage(sitk.ReadImage(n671_u16_file))
    n671_u16_mask = n671_u16_mask.astype(np.uint16)
    shape = n671_u16_mask.shape
    print(n671_u32_file,n671_u16_file,shape)

    n671_u32_voxel_dict = {}
    n671_u16_voxel_dict = {}
    n671_u32_center_dict = {}
    n671_u16_center_dict = {}
    n671_u32_neighbor_dict = {}    
    n671_u16_neighbor_dict = {}

    level_file = 'level.csv'
    level_df = pd.read_csv(level_file)
    u32_u16_dict_file = 'u32_u16_dict.json'
    u32_u16_id_dict = load_json_key2int(u32_u16_dict_file)
    print(f'iter id\nneighbors dict\nu25 voxel\nu25 center')
    n671_u32_id_list = []
    for i in level_df.index:
        n671_u32_id = int(level_df.iloc[i][f'n{cl}_u32_id'])
        if (not n671_u32_id) or (n671_u32_id in n671_u32_id_list): continue
        else: n671_u32_id_list.append(n671_u32_id)
        print('\n')
        
        mask_cur = n671_u32_mask == n671_u32_id
        mask_dil = skimage.morphology.binary_dilation(mask_cur, skimage.morphology.ball(radius=num_voxels))
        n671_u32_neighbor_dict[n671_u32_id] = [int(n) for n in np.unique(n671_u32_mask[mask_dil]) if n and n!=n671_u32_id]    
        n671_u32_center_dict[n671_u32_id] = get_center(mask_cur)        

        n671_u16_id_l = u32_u16_id_dict[n671_u32_id]['l']
        mask_left = mask_cur.copy()
        mask_left[shape[0]//2:] = False
        mask_dil = skimage.morphology.binary_dilation(mask_left, skimage.morphology.ball(radius=num_voxels))    
        n671_u16_neighbor_dict[n671_u16_id_l] = [int(n) for n in np.unique(n671_u16_mask[mask_dil]) if n and n!=n671_u16_id_l]    
        n671_u16_voxel_dict[n671_u16_id_l] = int(mask_left.sum())
        n671_u16_center_dict[n671_u16_id_l] = get_center(mask_left)
        
        n671_u16_id_r = u32_u16_id_dict[n671_u32_id]['r']
        mask_right = mask_cur.copy()
        mask_right[:shape[0]//2] = False   
        mask_dil = skimage.morphology.binary_dilation(mask_right, skimage.morphology.ball(radius=num_voxels))    
        n671_u16_neighbor_dict[n671_u16_id_r] = [int(n) for n in np.unique(n671_u16_mask[mask_dil]) if n and n!=n671_u16_id_r]   
        n671_u16_voxel_dict[n671_u16_id_r] = int(mask_right.sum())
        n671_u16_center_dict[n671_u16_id_r] = get_center(mask_right)

        n671_u32_voxel_dict[n671_u32_id] = n671_u16_voxel_dict[n671_u16_id_l] + n671_u16_voxel_dict[n671_u16_id_r]

        print(i,n671_u32_id,n671_u16_id_l,n671_u16_id_r)
        print(n671_u32_neighbor_dict[n671_u32_id],n671_u16_neighbor_dict[n671_u16_id_l],n671_u16_neighbor_dict[n671_u16_id_r])
        print(n671_u32_voxel_dict[n671_u32_id],n671_u16_voxel_dict[n671_u16_id_l],n671_u16_voxel_dict[n671_u16_id_r])
        print(n671_u32_center_dict[n671_u32_id],n671_u16_center_dict[n671_u16_id_l],n671_u16_center_dict[n671_u16_id_r])
    
    return n671_u32_id_list,n671_u32_voxel_dict,n671_u16_voxel_dict,n671_u32_center_dict,n671_u16_center_dict,n671_u32_neighbor_dict,n671_u16_neighbor_dict

In [ ]:
n671_u32_id_list,n671_u32_voxel_dict,n671_u16_voxel_dict,n671_u32_center_dict,n671_u16_center_dict,n671_u32_neighbor_dict,n671_u16_neighbor_dict = produce_level(671)

In [7]:
n316_u32_id_list,n316_u32_voxel_dict,n316_u16_voxel_dict,n316_u32_center_dict,n316_u16_center_dict,n316_u32_neighbor_dict,n316_u16_neighbor_dict = produce_level(316)

../../assets/n316_u32.nrrd ../../assets/n316_u16.nrrd (456, 320, 528)
iter id
neighbors dict
u25 voxel
u25 center


6 184 13 14
[723, 731, 746, 972, 993] [49, 477, 517, 529, 545] [50, 478, 518, 530, 546]
62227 31116 31111
[90.63, 110.84, 227.69] [90.63, 110.84, 182.38] [90.64, 110.84, 273.0]


18 985 37 38
[104, 337, 345, 361, 369, 894, 993, 1057] [49, 131, 145, 159, 173, 215, 559, 597] [50, 132, 146, 160, 174, 216, 560, 598]
726598 363311 363287
[171.84, 94.15, 227.7] [171.84, 94.16, 141.22] [171.84, 94.15, 314.18]


24 993 49 50
[39, 104, 184, 507, 723, 879, 894, 972, 985] [13, 37, 453, 477, 517, 559, 597, 651, 761] [14, 38, 454, 478, 518, 560, 598, 652]
838678 419476 419202
[144.16, 84.15, 227.68] [144.16, 84.15, 174.55] [144.15, 84.15, 280.85]


44 353 89 90
[329, 345, 378, 182305689] [103, 145, 187, 201] [104, 146, 188, 202]
193233 96621 96612
[226.8, 98.2, 227.7] [226.79, 98.2, 84.08] [226.8, 98.21, 371.33]


51 329 103 104
[353, 361, 378, 402, 417, 182305689, 312782546] [89, 173

346 312782546 693 694
[329, 361, 385, 394, 417, 533, 894] [103, 173, 343, 371, 399, 597, 707] [104, 174, 344, 372, 400, 598, 708]
92031 46020 46011
[289.93, 34.33, 227.7] [289.93, 34.35, 146.82] [289.92, 34.31, 308.58]


353 417 707 708
[329, 385, 402, 312782546] [103, 329, 371, 693] [104, 330, 372, 694]
65025 32516 32509
[311.83, 49.54, 227.69] [311.83, 49.54, 104.62] [311.83, 49.53, 350.77]


360 541 721 722
[378, 402, 677, 895, 1018, 1027, 312782574, 312782628] [201, 229, 287, 301, 329, 413, 427, 747] [202, 230, 288, 302, 330, 414, 428, 748]
198801 99536 99265
[331.03, 138.31, 227.39] [331.01, 138.34, 45.24] [331.06, 138.29, 410.03]


367 922 735 736
[111, 895, 918, 952, 312782628] [427, 571, 747, 989, 1119] [428, 572, 748, 990, 1120]
50565 25285 25280
[329.1, 180.06, 227.67] [329.08, 180.07, 41.05] [329.12, 180.05, 414.32]


373 895 747 748
[111, 541, 677, 918, 922, 952, 312782628] [229, 427, 571, 721, 735, 989, 1119] [230, 428, 572, 722, 736, 990, 1120]
109560 54766 54794
[321.22,

561 131 1123 1124
[295, 536, 672, 952, 966] [1119, 1121, 1125, 1147, 1191] [1122, 1126, 1148, 1192]
53680 26791 26889
[290.64, 209.28, 227.97] [290.65, 209.27, 83.42] [290.64, 209.3, 372.0]


562 295 1125 1126
[131, 319, 536, 566, 647, 780, 788, 961, 966, 1105] [833, 863, 889, 899, 1121, 1123, 1133, 1139, 1191, 1199] [834, 864, 890, 900, 1122, 1124, 1134, 1140, 1192, 1200]
121613 60829 60784
[288.4, 241.12, 227.65] [288.4, 241.12, 95.83] [288.39, 241.12, 359.56]


566 319 1133 1134
[23, 295, 403, 536, 566, 619, 639, 647, 780, 788, 966, 1105] [843, 855, 863, 889, 899, 1121, 1125, 1139, 1187, 1191, 1199, 1201] [844, 856, 864, 890, 900, 1122, 1126, 1140, 1188, 1192, 1200, 1202]
95068 47516 47552
[279.96, 255.22, 227.75] [279.96, 255.22, 114.33] [279.96, 255.21, 341.08]


569 780 1139 1140
[295, 319, 403, 566, 647, 726, 926, 589508447] [863, 899, 947, 1015, 1107, 1125, 1133, 1201] [864, 900, 948, 1016, 1108, 1126, 1134, 1202]
61821 30919 30902
[317.31, 248.82, 227.66] [317.3, 248.82, 124.5

651 563807435 1303 1304
[218, 325, 422, 475, 725, 733, 1020, 1029, 560581563] [1297, 1299, 1309, 1317, 1337, 1339, 1341, 1343, 1405] [1298, 1300, 1310, 1318, 1338, 1340, 1342, 1344, 1406]
17909 8941 8968
[322.51, 158.41, 227.81] [322.53, 158.42, 158.65] [322.48, 158.4, 296.76]


653 414 1307 1308
[609, 741, 946] [1301, 1311, 1585] [1302, 1312, 1586]
4449 2218 2231
[300.13, 191.37, 227.76] [300.11, 191.39, 215.63] [300.15, 191.35, 239.82]


654 422 1309 1310
[128, 725, 741, 560581563, 563807435] [1297, 1301, 1303, 1405, 1653] [1298, 1302, 1304, 1406, 1654]
8597 4304 4293
[317.18, 174.95, 227.62] [317.18, 174.96, 174.9] [317.18, 174.95, 280.47]


655 609 1311 1312
[59, 149, 414, 599, 741, 795, 930] [1301, 1307, 1312, 1369, 1385, 1397, 1403, 1677] [1302, 1308, 1311, 1369, 1370, 1385, 1386, 1398, 1404, 1678]
8454 4151 4303
[300.9, 168.7, 227.68] [300.93, 168.73, 222.55] [300.87, 168.67, 232.62]


656 1044 1313 1314
[66, 726, 560581563] [947, 1405, 1723] [948, 1406, 1724]
3932 1956 1976
[33

705 27 1411 1412
[170, 178, 563807439] [1325, 1413, 1415] [1326, 1414, 1416]
4415 2204 2211
[309.02, 148.38, 227.88] [309.03, 148.41, 125.12] [309.02, 148.36, 330.32]


706 563807439 1413 1414
[27, 170, 178, 475, 797, 560581563] [1317, 1405, 1411, 1415, 1605] [1318, 1326, 1406, 1412, 1416, 1606]
1448 715 733
[317.82, 162.1, 228.88] [317.83, 162.14, 133.35] [317.81, 162.06, 322.08]


707 178 1415 1416
[27, 170, 262, 321, 797, 563807439] [1325, 1407, 1411, 1413, 1421, 1605] [1326, 1408, 1412, 1414, 1422, 1606]
25892 12935 12957
[305.78, 156.97, 227.79] [305.79, 156.98, 121.29] [305.77, 156.95, 334.1]


710 321 1421 1422
[178, 262, 797] [1407, 1415, 1605] [1408, 1416, 1606]
1725 852 873
[305.8, 181.06, 228.95] [305.81, 181.08, 125.58] [305.79, 181.03, 329.83]


712 483 1425 1426
[149, 186, 362, 726, 795] [947, 1371, 1385, 1427, 1677] [948, 1372, 1386, 1428, 1678]
20492 10255 10237
[274.67, 120.96, 227.69] [274.71, 120.94, 218.33] [274.63, 120.98, 237.07]


713 186 1427 1428
[149, 362, 483

787 693 1575 1576
[118, 223, 614, 830] [1465, 1467, 1479, 1603] [1466, 1468, 1480, 1604]
35169 17556 17613
[272.12, 268.49, 227.71] [272.13, 268.5, 210.04] [272.12, 268.48, 245.32]


792 946 1585 1586
[118, 194, 414, 525, 795, 830, 980] [1307, 1465, 1479, 1541, 1561, 1586, 1589, 1677] [1308, 1466, 1480, 1542, 1562, 1585, 1590, 1678]
45251 22535 22716
[297.73, 223.04, 227.7] [297.7, 223.06, 215.41] [297.75, 223.03, 239.9]


794 194 1589 1590
[63, 88, 210, 226, 298, 342, 351, 356, 364, 390, 491, 523, 525, 614, 797, 946, 980, 576073704] [1227, 1229, 1243, 1435, 1527, 1529, 1541, 1561, 1565, 1585, 1591, 1593, 1595, 1597, 1603, 1605] [1228, 1230, 1244, 1436, 1490, 1516, 1528, 1530, 1542, 1562, 1566, 1586, 1592, 1594, 1596, 1598, 1604, 1606]
132570 66308 66262
[266.92, 239.23, 227.68] [266.92, 239.23, 179.07] [266.92, 239.23, 276.33]


795 226 1591 1592
[194, 263, 298, 342, 351, 523, 596, 689, 1109] [1227, 1229, 1237, 1243, 1475, 1489, 1495, 1589] [1228, 1230, 1238, 1244, 1476, 1490, 1496, 1

858 606826663 1717 1718
[115, 128] [1653, 1715] [1654, 1716]
1025 509 516
[383.65, 156.49, 227.78] [383.67, 156.51, 213.18] [383.63, 156.47, 242.17]


859 757 1719 1720
[128, 231] [1653, 1721] [1654, 1722]
2418 1196 1222
[396.72, 178.06, 227.81] [396.69, 178.07, 215.12] [396.76, 178.04, 240.23]


860 231 1721 1722
[128, 679, 757] [1653, 1719, 1851] [1654, 1720, 1852]
2757 1381 1376
[382.16, 185.99, 227.63] [382.16, 185.96, 217.38] [382.16, 186.01, 237.92]


861 66 1723 1724
[463, 797, 1044] [935, 1313, 1605] [936, 1314, 1606]
1147 570 577
[330.28, 177.51, 228.29] [330.32, 177.52, 132.16] [330.24, 177.5, 323.25]


862 75 1725 1726
[218, 294, 580] [1337, 1633, 1661] [1338, 1634, 1662]
725 363 362
[347.09, 125.22, 227.59] [347.06, 125.2, 147.82] [347.13, 125.23, 307.58]


863 58 1727 1728
[128, 374, 381, 749] [1645, 1647, 1653, 1733] [1646, 1648, 1654, 1734]
2753 1379 1374
[332.74, 212.4, 227.63] [332.73, 212.4, 186.22] [332.74, 212.41, 269.19]


866 374 1733 1734
[58, 128, 381, 749, 797]

944 711 1889 1890
[225, 429, 651, 859, 903, 1039, 1098] [1891, 1893, 1897, 1909, 1973, 1979, 2009] [1892, 1894, 1898, 1910, 1974, 1980, 2010]
21342 10684 10658
[511.08, 205.8, 227.67] [511.1, 205.79, 194.83] [511.07, 205.8, 260.59]


945 1039 1891 1892
[207, 651, 711] [1875, 1889, 1897] [1876, 1890, 1898]
5232 2611 2621
[519.11, 205.74, 227.73] [519.11, 205.74, 214.83] [519.12, 205.73, 240.59]


946 903 1893 1894
[225, 711, 765] [1889, 2009, 2013] [1890, 2010, 2014]
13130 6566 6564
[498.78, 193.63, 227.69] [498.78, 193.63, 173.62] [498.79, 193.64, 281.78]


947 642 1895 1896
[146, 398, 1093] [1805, 1827, 1863] [1806, 1828]
9895 4950 4945
[396.07, 270.25, 227.7] [396.06, 270.25, 201.67] [396.07, 270.25, 253.75]


948 651 1897 1898
[106, 136, 169, 202, 207, 225, 437, 445, 711, 839, 852, 859, 1039, 1098] [1875, 1876, 1889, 1891, 1898, 1911, 1913, 1947, 1957, 1959, 1973, 1977, 1979, 1993, 2007, 2009] [1876, 1890, 1892, 1897, 1912, 1914, 1948, 1958, 1960, 1974, 1978, 1980, 1994, 2008, 2010]

<ipython-input-6-570786759724>:4: RuntimeWarning: Mean of empty slice.
  cz = zcoords.mean().round(2)
/home/lyf/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-6-570786759724>:5: RuntimeWarning: Mean of empty slice.
  cy = ycoords.mean().round(2)
<ipython-input-6-570786759724>:6: RuntimeWarning: Mean of empty slice.
  cx = xcoords.mean().round(2)


985 395 1971 1972
[] [] []
0 0 0
[nan, nan, nan] [nan, nan, nan] [nan, nan, nan]


986 1098 1973 1974
[136, 235, 429, 437, 651, 711, 852] [1889, 1897, 1909, 1911, 1957, 1963, 1977] [1890, 1898, 1910, 1912, 1958, 1964, 1978]
69925 34955 34970
[510.87, 244.42, 227.72] [510.87, 244.41, 179.71] [510.88, 244.42, 275.7]


987 1107 1975 1976
[136, 177, 235, 773, 1048] [1951, 1957, 1963, 1991, 2015] [1952, 1958, 1964, 1992, 2016]
60123 30070 30053
[510.11, 259.21, 227.7] [510.11, 259.2, 202.87] [510.1, 259.22, 252.55]


988 852 1977 1978
[7, 106, 135, 136, 202, 203, 235, 429, 437, 445, 621, 651, 661, 978, 1098, 549009227] [1779, 1839, 1847, 1897, 1909, 1911, 1913, 1935, 1941, 1957, 1959, 1961, 1963, 1973, 1985, 2007] [1780, 1840, 1848, 1898, 1910, 1912, 1914, 1936, 1942, 1958, 1960, 1962, 1964, 1974, 1986, 2008]
143835 72030 71805
[457.96, 239.02, 227.58] [457.92, 239.03, 169.04] [458.01, 239.0, 286.3]


989 859 1979 1980
[202, 225, 651, 711] [1889, 1897, 2007, 2009] [1890, 1898, 2008, 2010]
1

In [ ]:
n671_u32_voxel_center_neighbor_dict_file = 'n671_u32_voxel_center_neighbor_dict.json'
n671_u32_voxel_center_neighbor_dict = dict([('n671_u32_voxel_dict',n671_u32_voxel_dict),
                                            ('n671_u32_center_dict',n671_u32_center_dict),
                                            ('n671_u32_neighbor_dict',n671_u32_neighbor_dict),])
write_json(n671_u32_voxel_center_neighbor_dict,n671_u32_voxel_center_neighbor_dict_file)


n671_u16_voxel_center_neighbor_dict_file = 'n671_u16_voxel_center_neighbor_dict.json'
n671_u16_voxel_center_neighbor_dict = dict([('n671_u16_voxel_dict',n671_u16_voxel_dict),
                                            ('n671_u16_center_dict',n671_u16_center_dict),
                                            ('n671_u16_neighbor_dict',n671_u16_neighbor_dict),])
write_json(n671_u16_voxel_center_neighbor_dict,n671_u16_voxel_center_neighbor_dict_file)

In [8]:
n316_u32_voxel_center_neighbor_dict_file = 'n316_u32_voxel_center_neighbor_dict.json'
n316_u32_voxel_center_neighbor_dict = dict([('n316_u32_voxel_dict',n316_u32_voxel_dict),
                                            ('n316_u32_center_dict',n316_u32_center_dict),
                                            ('n316_u32_neighbor_dict',n316_u32_neighbor_dict),])
write_json(n316_u32_voxel_center_neighbor_dict,n316_u32_voxel_center_neighbor_dict_file)

n316_u16_voxel_center_neighbor_dict_file = 'n316_u16_voxel_center_neighbor_dict.json'
n316_u16_voxel_center_neighbor_dict = dict([('n316_u16_voxel_dict',n316_u16_voxel_dict),
                                            ('n316_u16_center_dict',n316_u16_center_dict),
                                            ('n316_u16_neighbor_dict',n316_u16_neighbor_dict),])
write_json(n316_u16_voxel_center_neighbor_dict,n316_u16_voxel_center_neighbor_dict_file)

n316_u16_voxel_center_neighbor_dict= load_json_keys2int(n316_u16_voxel_center_neighbor_dict_file)
for k,v in n316_u16_voxel_center_neighbor_dict.items():
    print(type(k))
    for kk,vv in v.items():
        if type(vv)==list:
            print(type(kk),type(vv),type(vv[0]))
        else:
            print(type(kk),type(vv))
        break

<class 'str'>
<class 'int'> <class 'int'>
<class 'str'>
<class 'int'> <class 'list'> <class 'float'>
<class 'str'>
<class 'int'> <class 'list'> <class 'int'>


# history

In [ ]:
# def to_n671_dict(n1327_u32_id_list):
#     n1327_n671_dict = {}
#     for n1327_u32_id in n1327_u32_id_list:
#         t = u32_id_tree[n1327_u32_id]
#         n1327_u32_id_path = t['structure_id_path']
#         print(n1327_u32_id)
#         print(n1327_u32_id_path)
#         find = [i in n671_u32_id_list for i in n1327_u32_id_path]
#         print(find)
#         if sum(find[1:]): 
#             find.reverse()
#             n671_u32_id = n1327_u32_id_path[len(n1327_u32_id_path)-1-find.index(True)]
#         else:
#             n671_u32_id = 0
#         if n1327_u32_id==997: 
#             n671_u32_id=997
#         print(n671_u32_id)
#         n1327_n671_dict[n1327_u32_id] = n671_u32_id
#         print('\n')
#     print(len(n1327_n671_dict),n1327_n671_dict)
#     print(np.unique(list(n1327_n671_dict.values()),return_counts=True))
#     print(len(np.unique(list(n1327_n671_dict.keys()))) , len(np.unique(list(n1327_n671_dict.values()))))  
#     return n1327_n671_dict
# n1327_n671_dict = to_n671_dict(n1327_u32_id_list)

In [ ]:
# def to_n316_dict(n671_u32_id_list):
#     n671_u32_id_path_order_list = [u32_id_tree[i]['structure_id_path'].index(i) for i in n671_u32_id_list]
#     print(len(n671_u32_id_path_order_list),n671_u32_id_path_order_list)
#     order_path_order = sorted(range(len(n671_u32_id_path_order_list)),key=lambda x:n671_u32_id_path_order_list[x])
#     print(len(order_path_order),order_path_order,'\n')
#     n671_n316_dict = {}
#     for i in order_path_order:
#         n671_u32_id = n671_u32_id_list[i]
#         n671_u32_id_path = u32_id_tree[n671_u32_id]['structure_id_path']
#         print(n671_u32_id)
#         print(n671_u32_id_path)
#         find = [i in n316_u32_id_list for i in n671_u32_id_path]
#         print(find)
#         if sum(find):
#             find.reverse()
#             n316_u32_id = n671_u32_id_path[len(n671_u32_id_path)-1-find.index(True)]
#         else: 
#             n316_u32_id = 0
#         print(n316_u32_id)
#         n671_n316_dict[n671_u32_id] = n316_u32_id
#         print('\n')
#     print(len(n671_n316_dict),n671_n316_dict)
#     print(np.unique(list(n671_n316_dict.values()),return_counts=True))
#     print(len(np.unique(list(n671_n316_dict.keys()))) , len(np.unique(list(n671_n316_dict.values()))))
#     print([i for i in n316_u32_id_list if i not in np.unique(list(n671_n316_dict.values()))])
#     return n671_n316_dict
# n671_n316_dict = to_n316_dict(n671_u32_id_list)

In [ ]:
# n316_u32_id_path_order_list = [u32_id_tree[i]['structure_id_path'].index(i) for i in n316_u32_id_list]
# print(len(n316_u32_id_path_order_list),n316_u32_id_path_order_list)
# order_path_order = sorted(range(len(n316_u32_id_path_order_list)),key=lambda x:n316_u32_id_path_order_list[x])
# print(len(order_path_order),order_path_order,'\n')
# n70_u32_id_list = []
# n316_n70_dict = {}
# for i in order_path_order:
#     n316_u32_id = n316_u32_id_list[i]
#     n316_u32_id_path = u32_id_tree[n316_u32_id]['structure_id_path']
#     print(n316_u32_id)
#     print(n316_u32_id_path)
#     find = [i in n70_u32_id_list for i in n316_u32_id_path]
#     print(find)
#     if sum(find):
#         n70_u32_id = n316_u32_id_path[find.index(True)]
#     else:
#         n70_u32_id = n316_u32_id_path[-2]
#         if n70_u32_id==997: n70_u32_id = n316_u32_id
#         n70_u32_id_list.append(n70_u32_id)
#     print(n70_u32_id)
#     print(n70_u32_id_list)
#     print('\n')
#     n316_n70_dict[n316_u32_id] = n70_u32_id
# print(len(n316_n70_dict),n316_n70_dict)    
# print(len(np.unique(list(n316_n70_dict.keys()))) , len(np.unique(list(n316_n70_dict.values()))))
# print(len(n70_u32_id_list),n70_u32_id_list)

In [ ]:
# n5_u32_id_list, n5_u32_id_count_list = np.unique([t['structure_id_path'][1] for t in tree if len(t['structure_id_path'])>1],return_counts=True)
# n5_u32_id_list = n5_u32_id_list.tolist()
# n5_u32_id_count_list = n5_u32_id_count_list.tolist()
# print(len(n5_u32_id_list),n5_u32_id_list)
# print(len(n5_u32_id_count_list),n5_u32_id_count_list)

In [ ]:
# def to_n5_dict(n70_u32_id_list):
#     n70_n5_dict = {}
#     for i in n70_u32_id_list:
#         u32_id_path = u32_id_tree[i]['structure_id_path']
#         if len(u32_id_path)>1:
#             n70_n5_dict[i] = u32_id_path[1]
#         else:
#             n70_n5_dict[i] = 0
#     print(len(n70_n5_dict),n70_n5_dict)
#     print(np.unique(list(n70_n5_dict.values()),return_counts=True))
#     return n70_n5_dict
# n70_n5_dict = to_n5_dict(n70_u32_id_list)

In [ ]:
# n316_n5_dict = to_n5_dict(n316_u32_id_list)

In [ ]:
# n671_n5_dict = to_n5_dict(n671_u32_id_list)

In [ ]:
# n1327_n5_dict = to_n5_dict(n1327_u32_id_list)

In [ ]:
# def to_n70_dict(n671_u32_id_list):
#     n671_u32_id_path_order_list = [u32_id_tree[i]['structure_id_path'].index(i) for i in n671_u32_id_list]
#     print(len(n671_u32_id_path_order_list),n671_u32_id_path_order_list)
#     order_path_order = sorted(range(len(n671_u32_id_path_order_list)),key=lambda x:n671_u32_id_path_order_list[x])
#     print(len(order_path_order),order_path_order,'\n')
#     n671_n70_dict = {}
#     for i in order_path_order:
#         n671_u32_id = n671_u32_id_list[i]
#         n671_u32_id_path = u32_id_tree[n671_u32_id]['structure_id_path']
#         print(n671_u32_id)
#         print(n671_u32_id_path)
#         find = [i in n70_u32_id_list for i in n671_u32_id_path]
#         print(find)
#         if sum(find):
#             find.reverse()
#             n70_u32_id = n671_u32_id_path[len(n671_u32_id_path)-1-find.index(True)]
#         else: 
#             n70_u32_id = 0
#         print(n70_u32_id)
#         n671_n70_dict[n671_u32_id] = n70_u32_id
#         print('\n')
#     print(len(n671_n70_dict),n671_n70_dict)
#     print(np.unique(list(n671_n70_dict.values()),return_counts=True))
#     print(len(np.unique(list(n671_n70_dict.keys()))) , len(np.unique(list(n671_n70_dict.values()))))
#     print([i for i in n70_u32_id_list if i not in np.unique(list(n671_n70_dict.values()))]) 
#     return n671_n70_dict
# n671_n70_dict = to_n70_dict(n671_u32_id_list)

In [ ]:
# n1327_n70_dict = to_n70_dict(n1327_u32_id_list)

In [ ]:
# n1327_n316_dict = to_n316_dict(n1327_u32_id_list)

In [ ]:
# n316_u32_id_vol_list = (pd.read_csv('../../assets/CCFv3_316son_70parent_structures.csv')['total_voxel_counts (10 um)']*1000/15625).astype(int).tolist()
# n316_u32_vol_dict = dict(zip(n316_u32_id_list,n316_u32_id_vol_list))
# print(n316_u32_vol_dict)

In [ ]:
# n316_u32_voxel_dict,n316_u16_voxel_dict,n316_u32_center_dict,n316_u16_center_dict,n316_u32_neighbor_dict,n316_u16_neighbor_dict = produce_level(316)

In [ ]:
# n70_u32_voxel_dict,n70_u16_voxel_dict,n70_u32_center_dict,n70_u16_center_dict,n70_u32_neighbor_dict,n70_u16_neighbor_dict = produce_level(70)

In [ ]:
# n5_u32_voxel_dict,n5_u16_voxel_dict,n5_u32_center_dict,n5_u16_center_dict,n5_u32_neighbor_dict,n5_u16_neighbor_dict = produce_level(5)

In [ ]:
# new_tree = []
# for tt in range(len(tree)):
#     t = tree[tt]
    
#     new_t = {}
    
#     new_t['right'] = 0
#     new_t['name_lr'] = ['left ','right '][0] + t['name']
#     new_t['name'] = t['name']
#     new_t['acronym_lr'] = t['acronym'] + ['_l','_r'][0]
#     new_t['acronym'] = t['acronym']
#     new_t['rgb'] = t['rgb_triplet']
    
#     new_t['u16_id'] = 2*tt + 1
#     new_t['u32_id'] = t['id']
#     new_t['path_u32_id'] = t['structure_id_path']   
    
#     new_t['in_n671'] = 1 if new_t['u32_id'] in n671_u32_id_list else 0
#     new_t['n671_u32_id'] = n1327_n671_dict[new_t['u32_id']]
#     if new_t['n671_u32_id']: new_t['n671_u16_id'] = 2 *list(u32_id_tree.keys()).index(new_t['n671_u32_id']) + 1
#     else: new_t['n671_u16_id'] = 0  
#     new_t['in_n316'] = 1 if new_t['u32_id'] in n316_u32_id_list else 0
#     new_t['n316_u32_id'] = n1327_n316_dict[new_t['u32_id']]
#     if new_t['n316_u32_id']: new_t['n316_u16_id'] = 2 *list(u32_id_tree.keys()).index(new_t['n316_u32_id']) + 1
#     else: new_t['n316_u16_id'] = 0
#     new_t['in_n70'] = 1 if new_t['u32_id'] in n70_u32_id_list else 0
#     new_t['n70_u32_id'] = n1327_n70_dict[new_t['u32_id']]
#     if new_t['n70_u32_id']: new_t['n70_u16_id'] = 2 *list(u32_id_tree.keys()).index(new_t['n70_u32_id']) + 1
#     else: new_t['n70_u16_id'] = 0
#     new_t['in_n5'] = 1 if new_t['u32_id'] in n5_u32_id_list else 0
#     new_t['n5_u32_id'] = n1327_n5_dict[new_t['u32_id']]
#     if new_t['n5_u32_id']: new_t['n5_u16_id'] = 2 *list(u32_id_tree.keys()).index(new_t['n5_u32_id']) + 1
#     else: new_t['n5_u16_id'] = 0
    
#     new_t['voxel_1um_lr'] =   0
#     new_t['center_25um_lr'] = [0,0,0,]
#     new_t['neighbor_25um_lr'] = [0]
#     new_t['voxel_1um'] =   0
#     new_t['center_25um'] = [0,0,0,]
#     new_t['neighbor_25um'] = [0]    

#     new_t_right = new_t.copy()
#     new_t_right['right'] = 1
#     new_t_right['name_lr'] = ['left ','right '][1] + t['name']
#     new_t_right['acronym_lr'] = t['acronym'] + ['_l','_r'][1]
#     new_t_right['u16_id'] = 2*tt + 2
#     if new_t['n671_u16_id']: new_t_right['n671_u16_id'] = new_t['n671_u16_id'] + 1
#     if new_t['n316_u16_id']: new_t_right['n316_u16_id'] = new_t['n316_u16_id'] + 1
#     if new_t['n70_u16_id']: new_t_right['n70_u16_id'] = new_t['n70_u16_id'] + 1
#     if new_t['n5_u16_id']: new_t_right['n5_u16_id'] = new_t['n5_u16_id'] + 1    
    
#     new_tree.append(new_t)
#     new_tree.append(new_t_right)

In [ ]:
# for new_t in new_tree:
#     if new_t['in_n671']: 
#         new_t['voxel_1um_lr'] = n671_u16_voxel_dict[new_t['u16_id']]*25*25*25
#         new_t['center_25um_lr'] = n671_u16_center_dict[new_t['u16_id']]
#         new_t['neighbor_25um_lr'] = n671_u16_neighbor_dict[new_t['u16_id']]
#         new_t['voxel_1um'] = n671_u32_voxel_dict[new_t['u32_id']]*25*25*25
#         new_t['center_25um'] = n671_u32_center_dict[new_t['u32_id']]
#         new_t['neighbor_25um'] = n671_u32_neighbor_dict[new_t['u32_id']]      
    
#     if new_t['in_n316']: 
#         new_t['voxel_1um_lr'] = n316_u16_voxel_dict[new_t['u16_id']]*25*25*25
#         new_t['center_25um_lr'] = n316_u16_center_dict[new_t['u16_id']]
#         new_t['neighbor_25um_lr'] = n316_u16_neighbor_dict[new_t['u16_id']]
#         new_t['voxel_1um'] = n316_u32_vol_dict[new_t['u32_id']]*10*10*10
#         new_t['center_25um'] = n316_u32_center_dict[new_t['u32_id']]
#         new_t['neighbor_25um'] = n316_u32_neighbor_dict[new_t['u32_id']]         
    
#     if new_t['in_n70']: 
#         new_t['voxel_1um_lr'] = n70_u16_voxel_dict[new_t['u16_id']]*25*25*25
#         new_t['center_25um_lr'] = n70_u16_center_dict[new_t['u16_id']]
#         new_t['neighbor_25um_lr'] = n70_u16_neighbor_dict[new_t['u16_id']]
#         new_t['voxel_1um'] = n70_u32_voxel_dict[new_t['u32_id']]*25*25*25
#         new_t['center_25um'] = n70_u32_center_dict[new_t['u32_id']]
#         new_t['neighbor_25um'] = n70_u32_neighbor_dict[new_t['u32_id']]
           
#     if new_t['in_n5']: 
#         new_t['voxel_1um_lr'] = n5_u16_voxel_dict[new_t['u16_id']]*25*25*25
#         new_t['center_25um_lr'] = n5_u16_center_dict[new_t['u16_id']]
#         new_t['neighbor_25um_lr'] = n5_u16_neighbor_dict[new_t['u16_id']]
#         new_t['voxel_1um'] = n5_u32_voxel_dict[new_t['u32_id']]*25*25*25
#         new_t['center_25um'] = n5_u32_center_dict[new_t['u32_id']]
#         new_t['neighbor_25um'] = n5_u32_neighbor_dict[new_t['u32_id']]

In [ ]:
# def Test(n0='n671',n1='n316'):
#     test0 = [t[f'{n0}_u32_id'] for t in new_tree if t[f'in_{n0}']]
#     test1 = [t[f'{n1}_u32_id'] for t in new_tree if t[f'in_{n0}']]
#     test = dict(zip(test0,test1))
#     print(np.unique(list(test.keys()),return_counts=True))
#     print(np.unique(list(test.values()),return_counts=True))
#     print(len(np.unique(list(test.keys()))),len(np.unique(list(test.values()))))
#     return test
# test = Test('n70','n5')

In [ ]:
# def printtest(test,n671_n316_dict):
#     print(len(n671_n316_dict),n671_n316_dict)
#     print(np.unique(list(n671_n316_dict.values()),return_counts=True))
#     print(len(np.unique(list(n671_n316_dict.keys()))),len(np.unique(list(n671_n316_dict.values()))))
#     print(test==n671_n316_dict)
# printtest(test,n671_n70_dict)

In [ ]:
# def Test(n0='n671',n1='n316'):
#     test0 = [t[f'{n0}_u32_id'] for t in new_tree if t[f'{n0}_u32_id']]
#     test1 = [t[f'{n1}_u32_id'] for t in new_tree if t[f'{n0}_u32_id']]
#     test = dict(zip(test0,test1))
#     print(len(test),test)
#     print(np.unique(list(test.values()),return_counts=True))
#     print(len(np.unique(list(test.keys()))),len(np.unique(list(test.values()))))
#     return test
# test = Test('n671','n70')

In [ ]:
# def get_center_hemisphere(mask_left):
#     nz_left = torch.nonzero(mask_left).float()
#     zcoords_left = nz_left[:,0]
#     ycoords_left = nz_left[:,1]
#     xcoords_left = nz_left[:,2]
#     cz_left = zcoords_left.mean().item()
#     cy_left = ycoords_left.mean().item()
#     cx_left = xcoords_left.mean().item()
#     return cx_left, cy_left, cz_left


# center_file = 'region_centers_ccf25.csv'
# pdist_file = 'pdist_region_centers.csv'

# calc_type = 'region' # or region or pdist
# if calc_type == 'region':
#     print("calculating center")
#     id_mapper, id_rev_mapper = parse_id_map()
#     ana_dict = parse_ana_tree()

#     mask = load_image("../../assets/annotation_25.nrrd")
#     mask = torch.from_numpy(mask.astype(np.int64)).cuda()
#     centers = []
#     for i, idx in enumerate(torch.unique(mask)):
#         idx = idx.item()

#         if idx == 0:
#             continue

#         region_name = ana_dict[idx]['acronym']

#         mask_cur = mask == idx
#         mask_left = mask_cur.clone()
#         mask_left[mask.shape[0]//2:] = False
#         mask_right = mask_cur.clone()
#         mask_right[:mask.shape[0]//2] = False

#         idx_left_u16, idx_right_u16 = id_rev_mapper[idx]

#         cx_left, cy_left, cz_left = get_mask_hemisphere(mask_left)
#         cx_right, cy_right, cz_right = get_mask_hemisphere(mask_right)

#         if idx == 1051:
#             centers.append([idx_right_u16, idx, cx_right, cy_right, cz_right, 1, region_name])
#         else:
#             centers.append([idx_left_u16, idx, cx_left, cy_left, cz_left, 0, region_name])
#             centers.append([idx_right_u16, idx, cx_right, cy_right, cz_right, 1, region_name])

#         print(i)

#     centers_df = pd.DataFrame(centers, columns=['regionID', 'regionID_CCF', 'centerX', 'centerY', 'centerZ', 'right', 'region_name'])
#     centers_df.to_csv(center_file, sep=',', float_format='%.4f', index=False)

# elif calc_type == 'pdist':
#     df = pd.read_csv(center_file, usecols=[0,2,3,4], sep=',')
#     coords = df.to_numpy()[:,1:]
#     pdist = distance_matrix(coords, coords)
    
#     pdist = pd.DataFrame(pdist, columns=df['regionID']).set_index(df['regionID'])
#     pdist.to_csv(pdist_file, sep=',', float_format='%.4f')

In [ ]:
# def calc_distance(center_file,is_hemi,pdist_file,pdist_fig):
#     df = pd.read_csv(center_file, usecols=[2,3,4,6])
#     if is_hemi:
#         coords = df.to_numpy()[:,:-1]
#         pdist = distance_matrix(coords, coords)
#         pdist = pd.DataFrame(pdist, columns=df['acronym']).set_index(df['acronym'])
#     else:
#         coords = df.to_numpy()[::2,:-1]
#         pdist = distance_matrix(coords, coords)
#         pdist = pd.DataFrame(pdist, columns=df['acronym'][0::2]).set_index(df['acronym'][0::2])
        
#     pdist = pdist.fillna(0)
#     pdist.to_csv(pdist_file, sep=',', float_format='%.4f')

#     sns.heatmap(pdist)
#     plt.savefig(pdist_fig)
#     plt.close()
    
#     return pdist

# pdist_file = f'../data/pdist_n69region_centers_lr_ccfv25_reshape_{axis}.csv'
# pdist_fig = f'../plots/n69regions_centers_l_distance_{axis}.png'
# pdist = calc_distance(centers_file,is_hemi,pdist_file,pdist_fig) 

In [ ]:
# mask = np.array([mask]).astype(np.uint32)
# n671_u16_mask = np.array([n671_u16_mask]).astype(np.uint16)   
# n316_u32_mask = np.array([n316_u32_mask]).astype(np.uint32)   
# n316_u16_mask = np.array([n316_u16_mask]).astype(np.uint16)   
# n70_u32_mask = np.array([n70_u32_mask]).astype(np.uint32)   
# n70_u16_mask = np.array([n70_u16_mask]).astype(np.uint16)   
# n8_u32_mask = np.array([n8_u32_mask]).astype(np.uint32)   
# n8_u16_mask = np.array([n8_u16_mask]).astype(np.uint16)   

# save_image('../../assets/n671_u32.tiff',sitk.GetImageFromArray(mask)) 
# save_image('../../assets/n671_u16.tiff',sitk.GetImageFromArray(n671_u16_mask))
# save_image('../../assets/n316_u32.tiff',sitk.GetImageFromArray(n316_u32_mask))
# save_image('../../assets/n316_u16.tiff',sitk.GetImageFromArray(n316_u16_mask))
# save_image('../../assets/n70_u32.tiff',sitk.GetImageFromArray(n70_u32_mask))
# save_image('../../assets/n70_u16.tiff',sitk.GetImageFromArray(n70_u16_mask))
# save_image('../../assets/n8_u32.tiff',sitk.GetImageFromArray(n8_u32_mask))
# save_image('../../assets/n8_u16.tiff',sitk.GetImageFromArray(n8_u16_mask))

# mask = mask[0].astype(np.uint32)
# n671_u16_mask = n671_u16_mask[0].astype(np.uint16)   
# n316_u32_mask = n316_u32_mask[0].astype(np.uint32)   
# n316_u16_mask = n316_u16_mask[0].astype(np.uint16)   
# n70_u32_mask = n70_u32_mask[0].astype(np.uint32)   
# n70_u16_mask = n70_u16_mask[0].astype(np.uint16)   
# n8_u32_mask = n8_u32_mask[0].astype(np.uint32)   
# n8_u16_mask = n8_u16_mask[0].astype(np.uint16)   

# def plt_3d(mask):
#     print(mask.dtype)
#     fig,axes = plt.subplots(1,3,figsize=(20,20))
#     z,y,x = mask.shape
#     axes[0].imshow(mask[int(z/2),:,:],cmap='gray')
#     axes[1].imshow(mask[:,int(y/2),:],cmap='gray')
#     axes[2].imshow(mask[:,:,int(x/2)],cmap='gray')

#maskfile = '../../assets/n671_u32.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

# maskfile = '../../assets/n671_u32.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

# maskfile = '../../assets/n671_u16.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(n671_u16_mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

# maskfile = '../../assets/n316_u32.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(n316_u32_mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

# maskfile = '../../assets/n316_u16.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(n316_u16_mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

# maskfile = '../../assets/n70_u32.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(n70_u32_mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

# maskfile = '../../assets/n70_u16.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(n70_u16_mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

# maskfile = '../../assets/n8_u32.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(n8_u32_mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

# maskfile = '../../assets/n8_u16.nrrd'
# sitk.WriteImage(sitk.GetImageFromArray(n8_u16_mask), maskfile)
# test = sitk.GetArrayFromImage(sitk.ReadImage(maskfile))
# plt_3d(test)

In [ ]:
# [[type(t[tt]) for tt in t.keys()] for t in new_tree_test]

In [ ]:
# new_tree_test = new_tree.copy()
# for i,t in enumerate(new_tree_test):
#     print(i)
#     for tt in t.keys():
#         if type(t[tt])==np.int64:
#             print(tt)
#             new_tree_test[i][tt] = int(new_tree_test[i][tt])

In [ ]:
# tree_object = json.dumps(new_tree_test)
# with open("../../assets/new_tree.json", "w") as f:
#     f.write(tree_object)

In [ ]:
# with open("../../assets/new_tree.json", "r") as f:
#     test = json.loads(f.read())
# print(test)

In [ ]:
# [t for t in new_tree if t['acronym']=='ORB']